In [ ]:
def calibrate_main(xz):
    s_i_y=xz
    #s_i_y=s_i_y[0]
   
    keys=['e_rb','p_alfa','p_share_lump','p_gepar','p_gupar','p_chi','p_nw','p_nr','p_eta','p_gj','p_phi','p_mu','p_bb','p_A','p_theta','p_nu']
    e_rb,p_alfa,p_share_lump,p_gepar,p_gupar,p_chi,p_nw,p_nr,p_eta,p_gj,p_phi,p_mu,p_bb,p_A,p_theta,p_nu=[pars.get(key)[0] for key in keys]
    
    p_deltak=s_i_y*(e_rb)/(p_alfa-s_i_y) 
    s_k_L=(s_i_y/p_deltak)**(1/(1-p_alfa))      
    e_tau_L=(1-p_share_lump)*(p_gepar+p_gupar)/(1-p_alfa+e_rb*p_chi*s_i_y/p_deltak) 
    e_tau_S=p_chi*e_tau_L
    e_tau_scale=p_share_lump*(p_gepar+p_gupar)/(p_nw+p_nr) #this is the share of output charged as lump sum tax_ need to take it times output to find size of tax
    e_w=(1-p_alfa)*s_k_L**p_alfa
    s_y_L=(s_k_L)**p_alfa
    s_c_y=1-s_i_y-p_gepar-p_gupar
    
    etas=np.array(p_eta)     # This was there before but ignore for now. Not sure why it is there p_eta(any(p_gj>0,size(p_gj,1)))
    etas = etas[etas != 0]
    gjs=np.array(p_gj)             #ditto p_gj(any(p_gj>0,size(p_gj,1)))
    #education spending to output ratio
    gjs=np.array(p_gj)
    gjs = gjs[gjs != 0]
    s_e_y=(sum(etas*(gjs*p_gepar)**p_phi))**(1/p_phi)  #s_e_y=(sum(etas*(gjs*p_ge)**p_phi))**(1/p_phi)
    pars['p_deltak']=[p_deltak[0],'finds the depreciation rate of capital']
    pars['s_k_L']=[s_k_L,'finds the #capital labor ratio']
    pars['e_tau_L']=[e_tau_L,'labor tax rate']
    pars['e_tau_S']=[e_tau_S,'skilled labor tax rate']
    pars['e_tau_scale']=[e_tau_scale,'allows tax rates to differ but be in a fixed proportio...this is the share of output charged as lump sum tax_ need to take it times output to find size of taxn']
    pars['e_w']=[e_w,'output to labor ratio']
    pars['s_y_L']=[s_y_L,'output to labor ratio']
    pars['s_c_y']=[ s_c_y,'savings rate']
    pars['p_etas']=[etas,'??']
    pars['p_gjs']=[gjs,'???']
    pars['s_e_y']=[s_e_y,'???']
    
    #calls  a separate function, calibrate_vp, which finds vectors of hc/h0,
    #the discount rates (betas) and normalized labor hours taking p,e,s,a as inputs and returning updated p,e,s,a  
    #calc_v in turn calls three separate functions to aid in the
    #calculations: calibrate_vp_human_c , calibrate_vp_beta, calibrate_vp_labor
    
    calibrate_vp()
    
   
    keys=['e_l','s_h_h0','p_b','e_w','s_x_h0']
    e_l,s_h_h0,p_b,e_w,s_x_h0=[pars.get(key)[0] for key in keys]
     
    #Total labor
    e_L=(p_bb*(s_e_y*s_y_L)**p_mu*sum(e_l*s_h_h0))**(1/(1-p_mu))
                                                        
    zz=('zz',(s_e_y*s_y_L)**p_mu*sum(e_l*s_h_h0))
        #Total capital
    e_K=s_k_L*e_L
    #Total output
    e_Y=p_A*e_K**p_alfa*e_L**(1-p_alfa)
    #Total consumption
    e_C=s_c_y*e_Y
    #Total education expenditures
    e_e=s_e_y*e_Y
    
    e_tau_Lump=e_tau_scale*e_Y #this is the size of the lump sum tax where tau_scale is found above_ 
    
    
    #Initial consmuption
    st=np.arange(0,p_nw+p_nr,1)
    st2=np.arange(0,p_nw,1)
    e_c0=e_C/np.sum(((1+e_rb*(1-e_tau_S))**(st)*p_b)**(1/p_theta)) #note that this is written for more general case where p_theta ne 1
    #vector of consumptions
    e_c=e_c0*((1+e_rb*(1-e_tau_S))**st*p_b)**(1/p_theta)
                                    
    #Initial human capital
    e_h0=p_bb*e_e**p_mu
    #vector of human capitals
    e_h=s_h_h0*e_h0
    #vector of disutilities
    
    
    p_gama=e_w*e_h*(1-e_tau_L)/e_c[0:p_nw]/e_l**p_nu
    
   
        
    #create vector to fill with savings    
    e_s=np.empty((p_nw+p_nr))
    e_rev_l=np.empty((p_nw+p_nr))
    e_rev_s=np.empty((p_nw+p_nr))
    e_rev_lump=np.empty((p_nw+p_nr))
    
   
    #fill vector of savings
    for v in np.arange(0,p_nw+p_nr,1):
        
        if v==0:
            e_s[v]=e_w*e_h[v]*e_l[v]*(1-e_tau_L)-e_c[v]-e_tau_Lump 
            e_rev_l[v]=e_w*e_h[v]*e_l[v]*e_tau_L 
            e_rev_lump[v]=e_tau_Lump 
            #revenue is calculated for a later check
        elif v<p_nw: 
            e_s[v]=e_w*e_h[v]*e_l[v]*(1-e_tau_L)+(1+e_rb*(1-e_tau_S))*e_s[v-1]-e_c[v]-e_tau_Lump
            e_rev_s[v]=e_s[v-1]*e_rb*e_tau_S
            e_rev_l[v]=e_w*e_h[v]*e_l[v]*e_tau_L #revenue is calculated for a later check
            e_rev_lump[v]=e_tau_Lump 
        else:
            e_s[v]=(1+e_rb*(1-e_tau_S))*e_s[v-1]-e_c[v]-e_tau_Lump 
            e_rev_s[v]=e_s[v-1]*e_rb*e_tau_S #revenue is calculated for a later check
            e_rev_lump[v]=e_tau_Lump 
    
#Record some endgenous items
    e_Ge=p_gepar*e_Y
    e_Gu=p_gupar*e_Y
    p_x=e_h0*s_x_h0
    e_I=p_deltak*e_K
    e_ge=gjs*e_Ge 
    e_rk=e_rb+p_deltak
    e_Ges=1
    e_Gus=1
    e_GY=1
    e_Tot_hc=sum(s_h_h0*e_h0)
    e_Tot_labor=sum(e_l)
    e_rev_L=e_tau_L*e_L*e_w
    e_rev_S=e_tau_S*sum(e_s)*e_rb
    e_rev_Lump=sum(e_rev_lump)
    e_revenue=e_rev_S+e_rev_L+e_rev_Lump
    e_wg_hc=e_w*e_h0
    e_wg_L=e_w*e_L
    pars['e_K']=[e_K,'???']
    pars['e_L']=[e_L,'???']
    pars['e_e']=[e_e,'???']
    pars['e_hf']=[e_h0,'???']
    pars['e_Y']=[e_Y,'???']
    pars['e_w']=[e_w,'???']
    pars['e_rk']=[e_rk,'???']
    pars['e_rb']=[e_rb,'???']
    pars['e_Ge']=[e_Ge,'???']
    pars['e_Gu']=[e_Gu,'???']
    pars['e_C']=[e_C,'???']
    pars['e_I']=[e_I,'???']
    pars['e_Tot_hc']=[e_Tot_hc,'???']
    pars['e_Tot_labor']=[e_Tot_labor,'???']
    pars['e_GY']=[e_GY,'???']
    pars['e_Ges']=[e_Ges,'???']
    pars['e_Gus']=[e_Gus,'???']
    pars['e_tau_L']=[e_tau_L,'???']
    pars['e_tau_S']=[e_tau_S,'???']
    pars['e_tau_Lump']=[e_tau_Lump,'???']
    
    pars['e_rev_Lump']=[e_rev_Lump,'???']
    pars['e_rev_L']=[e_rev_L,'???']
    pars['e_rev_S']=[e_rev_S,'???']
    pars['e_rev_Lump']=[e_rev_Lump,'???']
    pars['e_wg_hc']=[e_wg_hc,'???']
    pars['e_wg_L']=[e_wg_L,'???']
    
    
    
    pars['p_x']=[p_x,'???']
    pars['e_I']=[e_I,'???']
    pars['e_ge']=[e_ge,'???']
    
    
    
    
   
    
    
   
    pars['e_revenue']=[e_revenue,'???']
    
    
    pars['p_gama']=[p_gama,'???']
    pars['e_s']=[e_s,'???']
    pars['e_h']=[e_h,'???']
    pars['e_c']=[e_c,'???']
    pars['e_l']=[e_l,'???']
    pars['e_rev_l']=[e_rev_l,'???']
    pars['e_rev_s']=[e_rev_s,'???']
    pars['e_rev_lump']=[e_rev_lump,'???']
    
    #do some checks___all should be 0__mostly helpful in writing and verifying code above
    a_check_check1=(sum(e_s)-e_K)
    a_check_check2=e_rk-p_alfa*(1/s_k_L)**(1-p_alfa)
    a_check_check3=(e_rev_L+e_rev_S+e_rev_Lump)/e_Y-p_gepar-p_gupar
    a_check_check4=(e_rev_L+e_rev_S+e_rev_Lump)-e_Ge-e_Gu
   
    #Capital market clearing, item to iterate on
    foc3=np.sum(e_s)-e_K
    
    return foc3
    

